In [1]:
using Test
using Random

using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
import TCIAlgorithms as TCIA
using ITensors
ITensors.disable_warn_order()
import QuanticsGrids: DiscretizedGrid, quantics_to_origcoord

In [3]:
fxy(x, y) = (exp(-0.4 * (x^2 + y^2)) + 1 + sin(x * y) * exp(-x^2) +
           cos(3 * x * y) * exp(-y^2) + cos(x + y)) + 0.05 * cos(1 / 0.001 * (0.2 * x - 0.4 * y)) + 0.0005 * cos(1 / 0.0001 * (-0.2 * x + 0.7 * y)) + 1e-5 * cos(1 / 1e-7 * (20 * x))

    R = 40
    grid = DiscretizedGrid{2}(R, (-5, -5), (5, 5))
    localdims = fill(4, R)

    qf = x -> fxy(quantics_to_origcoord(grid, x)...)

    tol = 1e-7

    pordering = TCIA.PatchOrdering(collect(1:R))

    creator = TCIA.TCI2PatchCreator(
        ComplexF64,
        qf,
        localdims;
        maxbonddim=50,
        rtol=tol,
        verbosity=1,
        ntry=10,
    )

    tree = TCIA.adaptivepartion(creator, pordering; verbosity=1, maxnleaves=1000)

iteration = 1, rank = 9, error= 2.9522096323120195e-7, maxsamplevalue= 4.050509999999999, nglobalpivot=5
iteration = 2, rank = 41, error= 3.984030360138169e-7, maxsamplevalue= 4.050509999999999, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration
iteration = 3, rank = 55, error= 0.005620730800734077, maxsamplevalue= 4.050509999999999, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration
iteration = 4, rank = 55, error= 0.005839847800480095, maxsamplevalue= 4.050509999999999, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration
iteration = 5, rank = 55, error= 0.0063220327275002475, maxsamplevalue= 4.050509999999999, nglobalpivot=5
others = [9, 41, 55, 55, 55]
Creating a task for [1] ...
Creating a task for [2] ...
Creating a task for [3] ...
Creating a task for [4] ...
iteration = 1, rank = 9, error= 3.189099927656359e-7, maxsamplevalue= 4.049938152365881, nglobalpivot=5
iteration = 1, rank = 9, error= 2.849836873153322e-7

Dict{Vector{Vector{Int64}}, TensorCrossInterpolation.TensorTrain{ComplexF64, 3}} with 16 entries:
  [[4], [4]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 24
  [[4], [2]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 43
  [[3], [3]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 25
  [[2], [1]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 30
  [[1], [1]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 24
  [[2], [3]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 38
  [[3], [4]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 30
  [[4], [1]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 38
  [[1], [3]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 43
  [[3], [2]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 39
  [[4], [3]] => TensorCrossInterpolation.TensorTrain{ComplexF64, 3} with rank 30
  [[2], [4]

In [4]:
using QuanticsTCI

In [8]:
qtci, ranks, errors = quanticscrossinterpolate(Float64, fxy, grid; tolerance=tol)

(QuanticsTCI.QuanticsTensorCI2{Float64}(TensorCrossInterpolation.TensorCI2{Float64} with rank 95, DiscretizedGrid{2}(40, (-5.0, -5.0), (5.0, 5.0), 2, QuanticsGrids.UnfoldingSchemes.fused, false), TensorCrossInterpolation.CachedFunction{Float64, UInt128} with 518624 entries), [29, 86, 95, 95, 95], [8.989842420693631e-8, 9.817314447459214e-8, 9.375537598179056e-8, 9.219902377394657e-8, 9.913626132862818e-8])